In [6]:
import openai
from openai import OpenAI
from dotenv import load_dotenv, dotenv_values 
from PIL import Image
from io import BytesIO
import time
import pandas as pd
import datetime

In [ ]:
#To run the streamlit code with upload enabled, run this command in a terminal - streamlit run chat.py --server.enableXsrfProtection false

In [7]:
# loading variables from .env file
#load_dotenv()

In [8]:
client = OpenAI()

In [9]:
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

ChatCompletionMessage(content="In the realm of code, a concept profound,\nRecursion reigns, in loops it is bound.\nA function that calls itself, again and again,\nIn a cycle of logic, like a mystical refrain.\n\nLike a mirror reflecting in endless array,\nRecursion unwinds, in a recursive display.\nEach iteration builds upon the last,\nUntil a base case is met at long last.\n\nInfinite loops, a danger it hides,\nBut with care and precision, it abides.\nDivide and conquer, its power revealed,\nIn algorithms and tasks, it's a mighty shield.\n\nSo remember this tale of recursion's grand art,\nIn the world of programming, a crucial part.\nA dance of function calls, in a mesmerizing direction,\nRecursion, a concept of endless reflection.", role='assistant', function_call=None, tool_calls=None)


In [10]:
# Upload a file with an "assistants" purpose
# file = client.files.create(
#   file=open("iris.csv", "rb"),
#   purpose='assistants'
# )

# file

In [11]:
from io import BytesIO  
#data = BytesIO(df.to_csv(index=False).encode('utf-8'))

file = client.files.create(
    file= BytesIO(pd.read_csv('Data/iris.csv').to_csv(index=False).encode('utf-8')),
    purpose='assistants'
)
file

FileObject(id='file-saMcVE3As4frIfC5tjoTVOXc', bytes=4009, created_at=1718586944, filename='upload', object='file', purpose='assistants', status='processed', status_details=None)

In [12]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="You are a data analyst. When asked question, write and run code to answer the question.",
  model="gpt-3.5-turbo",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Create a correlation plot of the data",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [13]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [14]:
print(run)

Run(id='run_ajcZgmYJDg2cr2zzgdf4JOxk', assistant_id='asst_Kxq0vzdsaYUqf6m1DvXJSUCv', cancelled_at=None, completed_at=None, created_at=1718586946, expires_at=1718587546, failed_at=None, incomplete_details=None, instructions='You are a data analyst. When asked question, write and run code to answer the question.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_9wddyFU0kLQmuh2TxJ8ErDyW', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, parallel_tool_calls=True)


In [15]:
while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        print(f"Current run status: {run.status}")
        if run.status == "completed":
                "Run complete"                
                break
        elif run.status == "failed":
                print("Failed")
                break
        
        time.sleep(10)

Current run status: queued
Current run status: in_progress
Current run status: completed


In [16]:
messages = client.beta.threads.messages.list( thread_id=thread.id)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_kqTCmuL43evgPLXF35YIjB2p', assistant_id='asst_Kxq0vzdsaYUqf6m1DvXJSUCv', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-cV5NvqQdH5AAd1lEh4PQVg9N'), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Here is the correlation plot of the data. The color intensity and the value displayed in each cell represent the degree of correlation between the corresponding pairs of variables:\n\n- Values close to 1 indicate a strong positive correlation.\n- Values close to -1 indicate a strong negative correlation.\n- Values close to 0 indicate a weak or no correlation.\n\nLet me know if you need any further analysis or visualization of the data.'), type='text')], created_at=1718586961, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ajcZgmYJDg2cr2zzgdf4JOxk', status=None, thread_id='thread_9wddyFU0kLQmuh2TxJ8ErDyW'), Me

In [17]:
image_file_id = ""
other_file_id=""
for message in messages:    
    print(message)
    if isinstance(message.content[0], openai.types.beta.threads.image_file_content_block.ImageFileContentBlock):
        image_file_id =  message.content[0].image_file.file_id   
        print('Found image file')
        #print("here",(message.content[0]).image_file.file_id)
    

Message(id='msg_kqTCmuL43evgPLXF35YIjB2p', assistant_id='asst_Kxq0vzdsaYUqf6m1DvXJSUCv', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-cV5NvqQdH5AAd1lEh4PQVg9N'), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Here is the correlation plot of the data. The color intensity and the value displayed in each cell represent the degree of correlation between the corresponding pairs of variables:\n\n- Values close to 1 indicate a strong positive correlation.\n- Values close to -1 indicate a strong negative correlation.\n- Values close to 0 indicate a weak or no correlation.\n\nLet me know if you need any further analysis or visualization of the data.'), type='text')], created_at=1718586961, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ajcZgmYJDg2cr2zzgdf4JOxk', status=None, thread_id='thread_9wddyFU0kLQmuh2TxJ8ErDyW')
Found image file
Message(id='msg_

In [18]:
image_data = client.files.content(image_file_id)
image_data_bytes = image_data.read()

im = Image.open(BytesIO(image_data_bytes))
im.show()
#im.show()

# with open("./my-image"+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+".png", "wb") as file:
#     file.write(image_data_bytes)

In [ ]:
im = Image.open('my-image2024_06_16_12_11_29.png')
im.show()

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)
for item in run_steps:
    print(item)

In [ ]:
def save_files_from_thread(thread_id): 
    messages = client.beta.threads.messages.list( thread_id=thread.id) 
    for message in messages: 
        annotations = message.content[0].text.annotations 
        if annotations: 
            for annotation in annotations: 
                print(annotation.type)
                if annotation.type == 'file_path': 
                    file_id = annotation.file_path.file_id 
                    # file_data = openai.File.retrieve(file_id) 
                    # with open(f"downloaded_files/{file_id}.txt", "wb") as file: 
                    #     file.write(file_data.content) 
                    #     print(f"File {file_id} saved successfully.")

save_files_from_thread(thread.id)